In [ ]:
import pandas as pd
import numpy as np

In [2]:
# Read excel
data = pd.read_excel('rollingsales_manhattan.xls', header = 4)

In [3]:
# Drop price <= 10,000
data = data[data['SALE PRICE'] > 10000]

In [4]:
# Turn sale month and neighbor to onehot encoder
neighbor = list(data.NEIGHBORHOOD.unique())
for name in range(len(neighbor)):
    data['if_' + neighbor[name]] = np.where(data['NEIGHBORHOOD'] == neighbor[name], 1, 0)
data['Sale_month'] = data['SALE DATE'].dt.month
for month in range(1,13):
    data['month_' + str(month)] = np.where(data['Sale_month'] == month, 1, 0)

In [5]:
# Combine Block and Lot
alist=[]
for i in data[["BLOCK","LOT"]].values:
    j=i[1]
    if len(str(j))==4:
        num=int(str(i[0])+str(j))
    else:
        for z in range(4-len(str(j))):
            newj="0"+str(j)
            j=newj
        num=int(str(i[0])+str(j))
    alist.append(num)
data["B+L"]=alist
blmin=np.min(data["B+L"])
blmax=np.max(data["B+L"])
data["BL STAN"]=list(map(lambda x: (x-blmin)/(blmax-blmin),data["B+L"]))
data=data.drop(["B+L"],axis=1)

In [6]:
# Split building class and address/apartment number
data['BUILDING CLASS'], data['BUILDING CLASS NO'] = 0, 0
for n in data.index:
    data.loc[n,'BUILDING CLASS'], data.loc[n,'BUILDING CLASS NO'] = data['BUILDING CLASS AT TIME OF SALE'][n][0], data['BUILDING CLASS AT TIME OF SALE'][n][1]
    if ',' in data['ADDRESS'][n]:
        data.loc[n,'ADDRESS'], data.loc[n,'APARTMENT NUMBER'] = data['ADDRESS'][n].split(',')[0], data['ADDRESS'][n].split(',')[-1]

In [7]:
# Tax class
tax_class = data['TAX CLASS AT TIME OF SALE'].unique().tolist()
for tax in tax_class:
    data['TAX CLASS = ' + str(tax)] = np.where(data['TAX CLASS AT TIME OF SALE'] == tax, 1, 0)

In [8]:
# Building class
RESIDENTIAL = ['A', 'B', 'C', 'D', 'R']
MIXED_USE = ['S']
COMMERCIAL_BUILDINGS = ['H','K','L','O']
INDUSTRIAL_BUILDINGS = ['E', 'F', 'G','T','U']
VACANT_LOTS = ['V']
CIVIC_USE = ['I','J','M','N','P','Q','W']
GOVERNMENTa_BLDG = ['Y']
OTHERS = ['Z']
class_dict = {'RESIDENTIAL': RESIDENTIAL, 'MIXED_USE': MIXED_USE, 'COMMERCIAL_BUILDINGS':COMMERCIAL_BUILDINGS, 
             'INDUSTRIAL_BUILDINGS':INDUSTRIAL_BUILDINGS, 'VACANT_LOTS':VACANT_LOTS, 'CIVIC_USE': CIVIC_USE, 
              'GOVERNMENT_BLDG':GOVERNMENTa_BLDG, 'OTHERS':OTHERS}
RESIDENTIAL_CLASS = {'One Family Dwellings': 'A','Two Family Dwellings': 'B','Walk Up Apartments':'C',
                     'Elevator Apartments':'D','Condominiums':'R'}
for key, value in class_dict.items():
    data[key] = np.where(data['BUILDING CLASS'].isin(value), 1, 0)
    if key == 'RESIDENTIAL':
        for key0, value0 in RESIDENTIAL_CLASS.items():
            data[key0] = np.where(data['BUILDING CLASS'] == value0, 1, 0)

In [9]:
# Help function
def MatFromList(match_list):
    for i in match_list:
        if i:
            return i
    return False

In [13]:
# Apartment Floor
import re
# Deal with unit style number+alphabet
pat_0A = r'^([0-9]{1,2})[A-Za-z]$'
# Deal with unit style: (alphabet)+3/4 numbers
pat_0000 = r'^[A-Z]?([0-9]{1,2})([0-9]{2})$'
# Deal with unit style numer and N/W/E/S/
pat_N0A = r'^[NSEW]([0-9]{1,2})[A-Za-z]$'
pat_A0N = r'^[A-Za-z]([0-9]{1,2})([NSEW])$'
pat_0AN = r'^([0-9]{1,2})[A-Za-z]([NSEW])$'
data['Apt Floor']=0

for idx,x in zip(data.index, data['APARTMENT NUMBER']):
    xs = x.replace('-','').replace('#', '').replace(' ', '').split('/')
    for x in xs:
        m1 = re.match(pat_0A, x)
        m2 = re.match(pat_0000, x)
        m3 = re.match(pat_N0A, x)
        m4 = re.match(pat_A0N, x)
        m5 = re.match(pat_0AN, x)
        match = MatFromList([m1, m2, m3, m4, m5])
        if match:
            data.loc[idx, 'Apt Floor'] = int(match.group(1))
data['Apt Floor_mv'] = (data['Apt Floor']==0).astype(int)
blmin=np.min(data['Apt Floor'])
blmax=np.max(data['Apt Floor'])
data["Apt Floor STAN"]=list(map(lambda x: (x-blmin)/(blmax-blmin),data['Apt Floor']))
data=data.drop(["Apt Floor"],axis=1)

In [14]:
# LAND SQUARE FEET: feed missing with mean and create a missing dummy
data['LAND SQUARE FEET_mv'] = (data['LAND SQUARE FEET']==0).astype(int)
mean_area = data['LAND SQUARE FEET'][data['LAND SQUARE FEET_mv']==0].mean()
data['LAND SQUARE FEET'][data['LAND SQUARE FEET_mv']==1]=mean_area
blmin=np.min(data["LAND SQUARE FEET"])
blmax=np.max(data["LAND SQUARE FEET"])
data["LAND SQUARE FEET STAN"]=list(map(lambda x: (x-blmin)/(blmax-blmin),data["LAND SQUARE FEET"]))
data=data.drop(["LAND SQUARE FEET"],axis=1)

/Users/jocelynlu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
# Years
year_list = [1800, 1900, 2000, 2019]
for i in range(len(year_list)-1):
    data['BUILD_YEAR('+str(year_list[i])+'-'+str(year_list[i+1])+')'] = np.where(data['YEAR BUILT'].isin(range(year_list[i], year_list[i+1])), 1, 0)

In [16]:
data = data.drop(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT', 'BLOCK', 
                  'LOT', 'EASE-MENT','BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
                  'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS','GROSS SQUARE FEET', 'YEAR BUILT',
                  'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE', 'SALE DATE', 'Sale_month', 
                  'BUILDING CLASS', 'BUILDING CLASS NO'], axis=1)
shuffled_data = data.loc[np.random.permutation(data.index), :]
shuffled_data

,SALE PRICE,if_ALPHABET CITY,if_CHELSEA,if_CHINATOWN,if_CIVIC CENTER,if_CLINTON,if_EAST VILLAGE,if_FASHION,if_FINANCIAL,if_FLATIRON,...,CIVIC_USE,GOVERNMENT_BLDG,OTHERS,Apt Floor_mv,Apt Floor STAN,LAND SQUARE FEET_mv,LAND SQUARE FEET STAN,BUILD_YEAR(1800-1900),BUILD_YEAR(1900-2000),BUILD_YEAR(2000-2019)
5864,990000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.125000,1,0.015092,0,1,0
9940,10750000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.000000,1,0.015092,0,1,0
5837,999000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.045455,1,0.015092,0,1,0
13365,825000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.261364,1,0.015092,0,0,0
3028,1150000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.056818,1,0.015092,0,1,0
12779,1297500,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.215909,1,0.015092,0,1,0
8561,435000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.022727,1,0.015092,0,1,0
15475,2675000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.113636,1,0.015092,0,1,0
9930,3400000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.056818,1,0.015092,0,0,0
4286,11500000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.000000,0,0.013476,0,1,0


In [17]:
# n_train = 0.7*len(data)
# n_valid = 0.2*len(data)
n_train = int(0.85*len(data))
n_test = int(0.15*len(data))

In [18]:
shuffled_data = data.loc[np.random.permutation(data.index), :]
X_train, y_train = shuffled_data.iloc[:n_train, :].drop(['SALE PRICE'], axis=1), shuffled_data.iloc[:n_train]['SALE PRICE']
X_test, y_test = shuffled_data.iloc[n_train:, :].drop(['SALE PRICE'], axis=1), shuffled_data.iloc[n_train:]['SALE PRICE']

In [19]:
data.columns

Index(['SALE PRICE', 'if_ALPHABET CITY', 'if_CHELSEA', 'if_CHINATOWN',
       'if_CIVIC CENTER', 'if_CLINTON', 'if_EAST VILLAGE', 'if_FASHION',
       'if_FINANCIAL', 'if_FLATIRON', 'if_GRAMERCY',
       'if_GREENWICH VILLAGE-CENTRAL', 'if_GREENWICH VILLAGE-WEST',
       'if_HARLEM-CENTRAL', 'if_HARLEM-EAST', 'if_HARLEM-UPPER',
       'if_HARLEM-WEST', 'if_INWOOD', 'if_JAVITS CENTER', 'if_KIPS BAY',
       'if_LITTLE ITALY', 'if_LOWER EAST SIDE', 'if_MANHATTAN VALLEY',
       'if_MIDTOWN CBD', 'if_MIDTOWN EAST', 'if_MIDTOWN WEST',
       'if_MORNINGSIDE HEIGHTS', 'if_MURRAY HILL', 'if_ROOSEVELT ISLAND',
       'if_SOHO', 'if_SOUTHBRIDGE', 'if_TRIBECA', 'if_UPPER EAST SIDE (59-79)',
       'if_UPPER EAST SIDE (79-96)', 'if_UPPER EAST SIDE (96-110)',
       'if_UPPER WEST SIDE (59-79)', 'if_UPPER WEST SIDE (79-96)',
       'if_UPPER WEST SIDE (96-116)', 'if_WASHINGTON HEIGHTS LOWER',
       'if_WASHINGTON HEIGHTS UPPER', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'mont

In [20]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
y_test_lr = lr.predict(X_test)